<a href="https://colab.research.google.com/github/binsalim95/binsalim95.github.io/blob/master/Dating_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dating App Recommender using Content Based and Collaborative Filtering

We'll find a guy for 10 girls using content based recommendations based on age income and substance use.

In [ ]:
#Dependencies
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# Load Data Set
df = pd.read_csv('okcupid_profiles.csv')
df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [ ]:
#Process Data, Get useful rows and columns
df_new = df[(df['orientation']=='straight') & (df['status']=='single')]
df_new = df_new[['age', 'sex','smokes', 'drinks', 'income', 'drugs']]
df_new.dropna(inplace=True)
df_new.head()

,age,sex,smokes,drinks,income,drugs
0,22,m,sometimes,socially,-1,never
1,35,m,no,often,80000,sometimes
4,29,m,no,socially,-1,never
7,31,f,no,socially,-1,never
9,37,m,no,not at all,-1,never


In [ ]:
#Create Map - smokes
keys = df_new['smokes'].unique()
values = [3,0,1,2,4]
smoke_map = dict(zip(keys, values))

print(keys)
print(values)
print(smoke_map)

['sometimes' 'no' 'trying to quit' 'when drinking' 'yes']
[3, 0, 1, 2, 4]
{'sometimes': 3, 'no': 0, 'trying to quit': 1, 'when drinking': 2, 'yes': 4}


In [ ]:
#Create Map - drinks
keys = df_new['drinks'].unique()
values = [2,3,0,1,5,4]
drink_map = dict(zip(keys, values))

print(keys)
print(values)
print(drink_map)

['socially' 'often' 'not at all' 'rarely' 'desperately' 'very often']
[2, 3, 0, 1, 5, 4]
{'socially': 2, 'often': 3, 'not at all': 0, 'rarely': 1, 'desperately': 5, 'very often': 4}


In [ ]:
#Create Map - drugs
keys = df_new['drugs'].unique()
values = [0,1,2]
drug_map = dict(zip(keys, values))

print(keys)
print(values)
print(drug_map)

['never' 'sometimes' 'often']
[0, 1, 2]
{'never': 0, 'sometimes': 1, 'often': 2}


In [ ]:
#Apply Map
df_new['smokes'] = df_new['smokes'].map(smoke_map)
df_new['drinks'] = df_new['drinks'].map(drink_map)
df_new['drugs'] = df_new['drugs'].map(drug_map)
df_new.head()

,age,sex,smokes,drinks,income,drugs
0,22,m,3,2,-1,0
1,35,m,0,3,80000,1
4,29,m,0,2,-1,0
7,31,f,0,2,-1,0
9,37,m,0,0,-1,0


In [ ]:
#Grab Males + IDs
males = df_new[df_new['sex'] == 'm']
males.drop(['sex'], axis=1, inplace=True)
m_list = males.index
males

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,age,smokes,drinks,income,drugs
0,22,3,2,-1,0
1,35,0,3,80000,1
4,29,0,2,-1,0
9,37,0,0,-1,0
18,31,0,1,-1,0
...,...,...,...,...,...
59932,21,4,2,-1,1
59939,52,0,2,-1,0
59942,24,0,3,-1,1
59943,42,0,0,100000,0


In [ ]:
#Grab 10 Females + IDs
females = df_new[df_new['sex'] == 'f'][:10]
females.drop(['sex'], axis=1, inplace=True)
f_list = females.index

In [ ]:
#Similarity Score
similarity_matrix = cosine_similarity(females,males)
similarity_matrix

array([[ 0.99049777, -0.03173569,  0.99998776, ...,  0.99731494,
        -0.03175556,  0.99655508],
       [-0.04381923,  0.99999984, -0.03337907, ..., -0.04027556,
         0.99999983, -0.03578746],
       [-0.04423569,  0.99999999, -0.03379967, ..., -0.04069492,
         0.99999999, -0.03620658],
       ...,
       [ 0.99087262, -0.04925188,  0.99940881, ...,  0.99880889,
        -0.049273  ,  0.99620054],
       [ 0.99080362, -0.03788131,  0.99996095, ...,  0.99801427,
        -0.03790162,  0.99660632],
       [ 0.9971424 , -0.03267704,  0.99387472, ...,  0.99088108,
        -0.03269608,  0.99639988]])

In [ ]:
#Putting Matches Together
couples = []
for i, a in enumerate(similarity_matrix):
  m = np.where(a == max(a))
  couple = [list(m_list[m])[0]]
  couple.append(f_list[i])
  couples.append(couple)

In [ ]:
print(couples)

[[660, 7], [2127, 13], [813, 14], [356, 17], [31, 20], [27, 21], [453, 32], [2449, 33], [356, 49], [27455, 67]]


In [ ]:
#Output Matches
for couple in couples:
  display(df.iloc[couple])
  print ('\n')

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
660,31,single,m,straight,fit,mostly anything,socially,never,graduated from masters program,white,...,"confident, genuine, and happy midwestern trans...",creative director and designer for a start-up ...,"problem solving. you name it, i can fix it, or...","physically: my blue eyes, my smile, or my ass....",i will give anything a first taste; just expec...,"adulation (love), ally (friends), miniature go...",...,"nothing about me or my life is typical, but i ...",i have no problem eating a pineapple if it lan...,"you desire to get to know me, and are availabl..."
7,31,single,f,straight,average,mostly anything,socially,never,graduated from college/university,white,...,NaN,"writing. meeting new people, spending time wit...","remembering people's birthdays, sending cards,...",i'm rather approachable (a byproduct of being ...,"i like: alphabetized lists, aquariums, autobio...","friends, family, notebook/pen, books, music, t...",things that amuse and inspire me,out and about or relaxing at home with a good ...,NaN,NaN


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
2127,30,single,m,straight,average,NaN,socially,never,working on two-year college,black,...,please dont let my name confuse you. im no pla...,im a property manager in san francisco and i f...,im good with computer hardware and troubleshoo...,im very observant. i see everything,"i like movies man on fire, forrest gump, the g...",my computer my tv my dog chicken family the sun,making my own business,at home if nothing is happening. im over the n...,i watched all of dawsons creek. =(,if your interested hit me up
13,30,single,f,straight,skinny,mostly anything,socially,never,graduated from high school,white,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
813,29,single,m,straight,a little extra,anything,socially,never,graduated from high school,hispanic / latin,...,i am a 28 year old mexican guy i am 6 feet tal...,i'm working and getting my life straight,anything i set my mind too. i'm good at footba...,my nice personality,my favorite movies would be the hangover part ...,"my iphone, music, my cars, football",life,hanging out with my cousins and brothers or go...,i'll tell u later,you would like to get to know a genuine guy th...
14,29,single,f,straight,thin,mostly anything,socially,never,working on college/university,"hispanic / latin, white",...,NaN,i have an awesome career working as a senior m...,listening. helping others. being patient. comm...,"well, i get the most compliments on my butt, s...",books: my all-time favorite book is george orw...,1. my family 2. italian or mexican food 3. my ...,"to be honest, i spend way too much time thinki...",unwinding from my work week. hanging out with ...,"hmmm...not sure if i want to admit this, but p...",...you genuinely think we'd be a good match.


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
356,26,single,m,straight,fit,mostly vegetarian,socially,never,graduated from college/university,white,...,a swirl of words i identify with: harpsichords...,i have a radio show (old school fm not interne...,making mix tapes finding rare music or films r...,i posed this question to my friends and they r...,"books: nonfiction: ""fooled by randomness"" (tal...",laughter my music gear/studio finding new stuf...,"time,conciseness, efficiency, how things were ...","at art murmur in oakland, at a friend's dance/...",i once had a mullet. i get vertigo in omni the...,points! a person's brain / their mental dispos...
17,26,single,f,straight,average,NaN,socially,never,working on college/university,hispanic / latin,...,to sum myself in whole: i have adventurous ten...,i'm currently studying and majoring in crimina...,hmm...i know i can take a test rather well.;) ...,physically-i've been told my eyes. people usua...,i'm much more of a movie goer. favorite actor:...,my cellphone eye liner food wallet jacket a go...,my next big vacation! who knows....maybe if i ...,"working out at the gym, sometimes watching a m...",i could beat you at a game of hold em'.....try...,you're interested? and you also thought ophel...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
31,27,single,m,straight,athletic,anything,socially,never,graduated from college/university,NaN,...,"to life, love and loot i like smart, funny, a...",im a founding partner and product manager at a...,making people laugh :) or making an ass out o...,my eyes. they're very green.,"i love tv, i love movies, and i love to read. ...",1. great food 2. big dreams 3. people smarter ...,"design, art, politics, movies, games, food!! ...",working with friends while having a beer tryin...,sometimes i'm still afraid of the dark. i've s...,if you like good conversation. it may not end...
20,27,single,f,straight,average,mostly anything,socially,never,graduated from college/university,white,...,i grew up in a small town in the midwest and h...,i work for an international packaging design a...,laughing? someone once told me i was an ego-bo...,NaN,NaN,"fam, friends, music, tacos, passport, nitrous ...",where will my next trip be? when will i grow u...,out with friends for happy hour which usually ...,i'm terrified of parallel parking,"you're awesome, have a sense of humor, think w..."


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
27,22,single,m,straight,average,anything,socially,never,NaN,asian,...,"i suck at these things, but here it goes. i'm ...",currently just working a full time job and han...,hmm i'm good at alot of things.,my spiky hair?,"i have read all the harry potter, lord of the ...",NaN,a little bit of everything! where im going to ...,if im not working im usually out hanging with ...,i have a okcupid account!,you want to get to know me better or hang out
21,22,single,f,straight,athletic,NaN,socially,never,working on college/university,NaN,...,my name is ashley and i live in san francisco....,i just graduated from usf with a major in inte...,NaN,"my smile. my eyes, and my hair apparently.","i'm not picky...but i tend to enjoy comedies, ...",NaN,NaN,going out in the city with friends.,NaN,-you like to be happy and have fun. -you are a...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
453,38,single,m,straight,athletic,mostly vegetarian,socially,never,NaN,white,...,"whoooaa, been in sf 6yrs, moved from london, l...","i am self employed general contractor, that ke...",thinking..,"is that an accent!? no i,m pretending to be fo...","star of the sea, off the grid ,long walk to fr...",good food friends family my health free time m...,what its all about,NaN,despite my alleged eco conciousness i just bou...,you wana
32,38,single,f,straight,average,mostly anything,socially,never,graduated from college/university,white,...,"much more to add, but this is a start... i am...",NaN,using spell check being a goofball. building r...,i'm guessing my hair,NaN,my how this has changed throughout the years.....,NaN,NaN,NaN,NaN


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
2449,20,single,m,straight,NaN,mostly anything,socially,never,working on college/university,white,...,"hey, i'm alex! i'm a bit of a geek, but i'm fa...",i'm spending the summer up here in the bay are...,"coming up with crazy ideas, thus the dreamer :)","my energetic personality, or that i really lik...","food: i'm up for trying almost anything, but i...",first and foremost is my family and friends. i...,"my future, how to make some of my ideas work, ...",either out with friends or staying in working ...,i think that a man's manners should include wh...,"you want to hang out or get to know me better,..."
33,20,single,f,straight,average,mostly vegetarian,socially,never,working on college/university,NaN,...,"""apparently,"" has become a new favorite word o...",working at a doggy daycare. it's really fun an...,being a girl... cooking vegetarian cuisine and...,people start speaking to me in spanish and i r...,"everything, yo. i really enjoy non-fiction stu...",i can't choose. please don't let me choose... ...,what daydreamers only think about. moving to ...,it varies! i work on my friday nights but if t...,i turn 21 on july 1st. give me money and gifts...,you wanna get guatemalan food with me in the m...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
356,26,single,m,straight,fit,mostly vegetarian,socially,never,graduated from college/university,white,...,a swirl of words i identify with: harpsichords...,i have a radio show (old school fm not interne...,making mix tapes finding rare music or films r...,i posed this question to my friends and they r...,"books: nonfiction: ""fooled by randomness"" (tal...",laughter my music gear/studio finding new stuf...,"time,conciseness, efficiency, how things were ...","at art murmur in oakland, at a friend's dance/...",i once had a mullet. i get vertigo in omni the...,points! a person's brain / their mental dispos...
49,26,single,f,straight,athletic,mostly anything,socially,never,graduated from college/university,white,...,NaN,NaN,NaN,NaN,NaN,concerts running hanging out with friends and ...,NaN,NaN,NaN,NaN


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
27455,30,single,m,straight,used up,anything,rarely,sometimes,graduated from masters program,white,...,mostly water,grasping at straws,"this new sport i invented, citroll. you try it...",i'm singing in a quavering falsetto,"delillo, dfw, thomas pynchon, jonathan lethem,...","rza, gza, ghostface, raekwon, odb, and method ...",signal chains,playing a show or recording or making software...,"really more of a neil fan, actually. but yeiln...","you are like, articulate and stuff."
67,30,single,f,straight,thin,NaN,rarely,sometimes,working on college/university,"black, native american, white",...,"above all else i am, and always have been, a g...",i am currently studying early childhood develo...,"always being true to myself, and making no apo...",i dunno... a lot of people point out my good p...,books: lately a lot of my reading has been for...,"love hope faith family (no matter what i try, ...",how i should not spend so much time in my head...,"usually i am studying, working on any number o...",i suffer from a debilitating disorder that for...,you are compelled to in any way and want to ge...


## Using Collaborative Recommendations

We'll find a girl for 10 guys with collaborative filtering.



In [ ]:
#Dependencies
import seaborn as sns
from sklearn.cluster import KMeans 

In [ ]:
# Cluster Guys profiles
model = KMeans(10)
model.fit(males)

KMeans(n_clusters=10)

In [ ]:
#Update df
males['cluster'] = model.predict(males)
males['id'] = m_list
males.groupby(['cluster']).size()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


cluster
0    14610
1      230
2      927
3      102
4      778
5      385
6       28
7     1226
8      971
9      944
dtype: int64

In [ ]:
#Picking Guy from each Cluster
bachelors = males.groupby(['cluster']).first()
bachelors

,age,smokes,drinks,income,drugs,id
cluster,,,,,,
0,22,3,2,-1,0,0
1,41,0,2,1000000,2,102
2,34,0,2,100000,0,82
3,29,2,3,250000,0,177
4,24,0,2,60000,0,145
5,35,0,2,150000,1,72
6,21,0,1,500000,0,107
7,22,0,1,20000,0,66
8,35,0,3,80000,1,1


In [ ]:
#Removing Bachelor above from his cluster
guy_group = pd.merge(males, bachelors, how='outer', indicator=True)
new_guy_group = guy_group.loc[guy_group._merge == 'left_only']
new_guy_group.drop(['_merge'], axis=1, inplace =True)
new_guy_group.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,age,smokes,drinks,income,drugs,cluster,id
2,29,0,2,-1,0,0,4
3,37,0,0,-1,0,0,9
4,31,0,1,-1,0,0,18
5,33,0,2,-1,0,0,19
6,30,0,2,-1,0,0,22


In [ ]:
#Assemble All girls 
females = df_new[df_new['sex'] == 'f']
females.drop(['sex'], axis=1, inplace=True)
f_list = females.index

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#Similarity Matrix
similarity_matrix = cosine_similarity(new_guy_group.iloc[:, :5],females)
similarity_matrix

array([[ 0.99998776, -0.03337907, -0.03379967, ...,  0.99992495,
         0.99998776,  0.99923678],
       [ 0.99791383, -0.02601751, -0.02643737, ...,  0.99673665,
         0.99791383,  0.99937541],
       [ 0.99948281, -0.03122342, -0.03164386, ...,  0.99884058,
         0.99948281,  0.99988173],
       ...,
       [ 0.99731494, -0.04027556, -0.04069492, ...,  0.99815817,
         0.99731494,  0.99476881],
       [-0.03175556,  0.99999983,  0.99999999, ..., -0.03942262,
        -0.03175556, -0.01651729],
       [ 0.99655508, -0.03578746, -0.03620658, ...,  0.99658976,
         0.99655508,  0.9956137 ]])

In [ ]:
groups = list(new_guy_group.groupby(['cluster']).size())
groups.insert(0,0)
print(groups)

[0, 14609, 229, 926, 101, 777, 384, 27, 1225, 970, 943]


In [ ]:
#Sectioning off the scores of each cluster
grouping = []
for i in range(len(groups)-1):
  group = similarity_matrix[groups[i]:groups[i] + groups[i+1]]
  grouping.append(group)


In [ ]:
print (len(grouping))

10


In [ ]:
# Decide which girls each group liked by finding the maximum of their average rating
couples = []
for women in grouping:
  avg = women.mean(axis=1)
  best = np.where(avg == max(avg))
  couples.append(list(f_list[best]))
  

In [ ]:
#Putting Couples together by couple ID
for i in range(len(couples)):
  couples[i].append(bachelors['id'][i])

In [ ]:
couples

[[10317, 0],
 [205, 102],
 [952, 82],
 [346, 177],
 [238, 145],
 [472, 72],
 [89, 107],
 [460, 66],
 [317, 2394, 1],
 [1491, 3246, 94]]

In [ ]:
# Show pairings
for couple in couples:
  display(df.iloc[couple])
  print ('\n')

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
10317,46,single,f,straight,curvy,NaN,socially,never,NaN,black,...,about me? well it's really hard to describe on...,i'm at very confortable state in my life right...,cherishing and fostering friendship! i love pe...,"my smile, openness, friendliness, cheerfulness.","philosophy books, action movies, alternative r...","family, friends, faith, music, good health, an...",family and friends and travling.,hanging out with my friends or entertaining my...,i'm not saying any private thing (lol),you are looking for a real friendship!
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
205,43,single,f,straight,curvy,strictly anything,socially,never,graduated from law school,white,...,"i'm a writer. i do other stuff for a paycheck,...",i have a job. i'm good at it. and i write beca...,"having adventures, learning languages, helping...",that i look a lot younger. my hair. my ability...,"writers: alice munro, tom robbins, kazuo ishig...","books, paper, music, friends, socks, peppermin...","motivation, perspective, fresh peaches and hum...",writing or sleeping or out with friends or som...,i have blonde eyebrows. i am excellent with ey...,"you're creative, funny, kind, honest, adventur..."
102,41,single,m,straight,athletic,mostly anything,socially,often,working on space camp,white,...,i'm on a temporary hiatus from trying to defin...,"for a living? teaching cpr, first aid, and ass...",being unequivocal... sometimes. public speaki...,that my green eyes are red.,"when it comes to fiction, i like to read what ...","someday, i'll be a monk on a hill and all i'll...",human behavior. my goals. why i'm not further ...,falling on my ass a hundred times (that means ...,how about some dealbreakers right up front... ...,you like my words. you like my face. you'd lik...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
952,26,single,f,straight,thin,strictly anything,socially,sometimes,graduated from college/university,asian,...,"i'm warm, artistic, funny, and thoughtful. i'm...",right now i sling expensive shoes to gullible ...,thrifting. eating obscene amounts of food. mak...,"if they're showing, my tattoos.","books: comic books, a song of ice and fire, da...",looking at stupid and awesome shit on the inte...,NaN,NaN,i'm clumsy. embarrassingly so. like grownass a...,"we have some stuff in common, you aren't lame,..."
82,34,single,m,straight,fit,anything,socially,never,graduated from masters program,white,...,"i moved to sf ~7 months ago, and so far i'm re...",isn't this the million dollar question... work...,"solving problems, whatever life throws at me, ...","my blue eyes, and that i look rather young for...",adventure and scifi books thrillers and comedy...,my passport a playful pet comfortable bed good...,"where i'm traveling to next, and where do i fi...","i typically don't like ""typical"" nights... i'l...",i'm actually very open about things that perha...,"if you love hiking/camping, traveling in 3rd-w..."


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
346,43,single,f,straight,fit,strictly anything,socially,never,NaN,white,...,i told my brother-in-law (it guy) that i didn'...,working hard and working on playing hard harder.,finding the humor in things and being super ch...,what they tell me is that they think i am funn...,"movies, i love movies and buttered popcorn.......",or i don't want to anyway.... strong coffee fr...,"what sounds good to eat, when can i go on vaca...",either out on the town or on the couch.,NaN,you feel compelled to do so.
177,29,single,m,straight,a little extra,mostly halal,often,never,dropped out of high school,white,...,you might be wondering why my profile name is ...,being a bad ass mutha fucka,-eating -drinking -sex in that order...don't ...,-my hairy hairy chest -my strange stench...dai...,food...all the rest of that is bullshit. see a...,hmm...for all you lame-o romantics out there: ...,NaN,-learning how to dive -working out at the gym ...,sometimes i jack off while watching mad men......,"you're a girl, you're hot, you're funny, you l..."


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
238,22,single,f,straight,average,strictly vegetarian,socially,never,graduated from high school,white,...,new to the online dating and here to check thi...,currently i am a full time hairstylist as well...,"educating, working hard and painting.",my hair usually is the first thing people noti...,"anything by palahniuk, orwell, huxley, vonnegu...",1. my cat baloo 2. my records 3. good food 4. ...,NaN,usually hanging out with friends or my coworke...,i crush on iggy pop.,you like eccentric personalities.
145,24,single,m,straight,skinny,NaN,socially,never,graduated from college/university,"hispanic / latin, white",...,i just moved to sf from cincinnati. i was teac...,i just finished two years as a high school mat...,"listening, making people feel better, origami,...","my friendly demeanor, i always try to greet pe...","books: anything by mitch albom, harry potter, ...","computer/internet, my dog though he had to sta...","life, the universe, and everything. also not s...",watching college football at a bar; hanging ou...,i watch and enjoy the tv show glee. since ther...,anything on this profile made you smile. addi...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
472,47,single,f,straight,fit,mostly vegetarian,socially,never,graduated from college/university,white,...,what would a match for me be like? given that ...,"creative director in the tech realm, raising m...",having fun and being a friend. hopefully there...,i'm interested in them.,i'll get back to you on that...too many to list.,hugs music friends and family laughter hot tea...,where the world is headed politically and cult...,having dinner with friends.,"that essentially i am a private person, thus p...","if you live in my proximity and you read my ""e..."
72,35,single,m,straight,athletic,mostly anything,socially,sometimes,graduated from space camp,"native american, white",...,"i'm an adventurer first, i take calculated ris...","photography, lights, and sound. i also throw s...",pretty much anything i put my mind to... relat...,i have laser beams coming out of my eyes.,NaN,"my camera, aviation, amazing food, touch, lear...","travel, food, photography, sensual fun, aviati...","often working a party. lights, sound, managing...",NaN,you can make me laugh or turn me on. teach me ...


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
89,37,single,f,straight,average,mostly anything,socially,never,graduated from college/university,NaN,...,"i'm a pisces, odd because i'm allergic to fish...",i'm a working professional who loves their job...,gut feelings. intuition. making people feel at...,i don't know. you tell me... if i had to guess...,the alchemist. the red tent. a tree grows in b...,1. sunshine 2. wine 3. apples and tomatoes 4. ...,"i think a lot. sometimes it's random, other ti...",either at a happy hour and grabbing a bite to ...,nada,NaN
107,21,single,m,straight,athletic,mostly anything,rarely,never,working on med school,white,...,"hi, im sean, so these sites are always a wired...","still going to school, but am currently workin...","talking to anyone, im pretty easy going!",im nice and easy to get along with,NaN,being happy my phone my family my friends my life,NaN,"out with the guys, figuring out something to d...",NaN,"you read all the way down to here, say somethi..."


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
460,22,single,f,straight,average,mostly anything,socially,never,working on college/university,"asian, pacific islander",...,we only live once and thats why i live life to...,"right now i'm a full time student, majoring in...","hmm.. observing people, listening, giving advi...",the first thing people notice about me i would...,"i have a lot of favorite books, it changes thr...",- family& friends - music - sketchbook - iphon...,how i'll attain my goals and where i'll travel...,"it depends, i'm either with friends catching u...",nothing haha get to know me to find out ;),- if you want to get to know me - want to be f...
66,22,single,m,straight,athletic,mostly anything,rarely,never,working on college/university,asian,...,it will be way too boring to just reveal yours...,work+school,making bad jokes and being sarcastic,NaN,"i like christopher nolan movies,,,and amelie",cetaphil face wash phone internet family friends,my life goals and a career,NaN,NaN,just say hi..i dont bite


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
317,29,single,f,straight,curvy,mostly vegetarian,socially,never,working on college/university,"black, native american",...,"if you are anything like me, you probably look...",i am a student and my major is business market...,NaN,my smile.,"movies: truthfully, i love movies and there ar...",the internet. a roof over my head. my friends ...,my future and what kind of life i want to have.,at home or with friends. i am over the bar scene.,NaN,if you don't have a problem with other races a...
2394,37,single,f,straight,fit,mostly anything,socially,never,graduated from college/university,white,...,"i grew up an air force brat, moving every 3-4 ...",trying to achieve a certain balance... to live...,"coming up with witty user names, laughing and ...",my smile,i love all food... (well almost) and i love to...,"live music, yoga, the ocean, family, great fri...","how to get more out of my day, to do and exper...","grabbing cocktails and dinner with friends, lo...",i still really like coldplay.,you like to have fun and can make me laugh. yo...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
1491,40,single,f,straight,athletic,NaN,socially,never,graduated from college/university,white,...,i am not very good about writing about myself ...,"having just turned 40, i have a new lease on l...",i don't know that i would go as far as saying ...,my smile. i love laughing and being happy and ...,i do try to mix up my reading with murder/myst...,my friends and family. they mean the world to ...,NaN,NaN,we can discuss later.,NaN
3246,27,single,f,straight,fit,strictly vegetarian,socially,never,graduated from law school,white,...,some people tend to think i'm very serious whe...,"i'm a fairly new attorney, working in higher e...",figuring out personalities quickly.,my smile.,i love to read! anything and everything. one o...,"friends, family, beach, books, food, and animals","anything and everything. people, politics, it ...",varies.,i'm an open book. there's nothing that i am em...,"you are: open-minded, kind, witty, easy-going,..."
94,29,single,m,straight,fit,mostly anything,socially,sometimes,graduated from college/university,white,...,"my names josh, and i create art for a living. ...",living it,everything,i honestly couldn't say....,-books: anything joseph campbell - osho - terr...,invalid question,the world,out.,no,you're curious.
